In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from augur.utils import json_to_tree
import json
import math

# Grabbing accumulated mutations for each variant

In [24]:
# There are multiple internal nodes with value BA.2
END_NODE = "NODE_0000260"
#END_NODE = ["NODE_0000260", "NODE_0000261", "NODE_0000265","NODE_0000267","NODE_0000271","internal_c1ab","NODE_0000284","NODE_0000285"]

#grab lineage of tip node
def find_lineage(tree, child_node, rows_for_tsv):
    gene_muts = {}
    node_path = tree.get_path(child_node)
    # remove all elements of path before terminal node if it exists
    end_node = [j for j in node_path if j.name in END_NODE]
    if len(end_node) != 0:
        end_node = end_node[0]
        node_path = node_path[node_path.index(end_node)+1:]
       # do not include child node in path
        i = len(node_path) - 1
        # add all mutations in split path to list
        while (i >= 0):
            gene_muts = format_string(node_path[i].branch_attrs['mutations'],'nuc', gene_muts)
            i = i - 1
        #append variant name, mutation, and region to list
        for k in gene_muts.keys():
            for j in range(len(gene_muts[k])):
                temp_dict = gene_muts[k]
                mut =  str(temp_dict[j]) + '_' + str(k)
                rows_for_tsv.append({'variant': child_node.name, 'mutation': mut})

#put all the same gene mutations in the same list
def format_string(dict, key, gene_muts):
    for k in dict.keys():
        if (str(k) != key):
            if k not in gene_muts.keys():
                gene_muts[k] = []
            for j in range(len(dict[k])):
                temp_dict = dict[k]
                gene_muts[k].append(temp_dict[j])
    return gene_muts

if __name__ == '__main__':
    #read in tree
    with open(f'../mutations-by-variant/pango_lineages.json', 'r') as f:
        tree_json = json.load(f)
    #put tree in Bio.Phylo format
    tree = json_to_tree(tree_json)
    #create list of dictionaries for dataframe
    rows_for_tsv = []
    for tip in tree.find_clades(terminal=True):
        #append new row for every mutation in every child node
        find_lineage(tree, tip, rows_for_tsv)
    #make pandas dataframe from list of dicts
    accumulated_mutations = pd.DataFrame(rows_for_tsv)

In [25]:
#viewing accumulated mutations df
accumulated_mutations.to_csv("accumulated_mutations_output.tsv", sep="\t")
accumulated_mutations

,variant,mutation
0,BA.2.15,T3150I_ORF1a
1,BA.2.15,W64L_S
2,BA.2.16,T296I_N
3,BA.2.16,V84-_ORF1a
4,BA.2.16,M85-_ORF1a
...,...,...
5167,CK.2.1.1,V62F_E
5168,CK.2.1.1,T1050N_ORF1b
5169,CK.2.1.1,D16G_ORF9b
5170,CK.2.1.1,D3N_M


# Assigning Mutational GAs to each variant

In [4]:
import jax.numpy as jnp
import numpy as np
import evofr as ef
from evofr.models.mutational_fitness_mlr import MutationalFitnessMLR, MutationalFitnessSequenceCounts

In [27]:
# Getting data
raw_seq = pd.read_csv("../../sars2-epistasis-modeling/count-data/pango_location-variant-sequence-counts.tsv", sep="\t")
raw_seq = raw_seq[~raw_seq.variant.str.startswith("X")]
raw_muts = accumulated_mutations
variant_frequencies = MutationalFitnessSequenceCounts(raw_seq, raw_muts)

In [28]:
# Defining model
mlr = MutationalFitnessMLR(tau=4.2)
# Defining inference method
inference_method = ef.InferMAP(iters=100_000, lr=4e-4)
# Fitting model
posterior = inference_method.fit(mlr, variant_frequencies)
samples = posterior.samples

ValueError: MultinomialProbs distribution got invalid probs parameter.

In [7]:
# defining the growth advantages
mut_ga = pd.DataFrame(
    ef.posterior.get_growth_advantage(samples, posterior.data, ps=[0.8], name="USA", rel_to="BA.2")
)

In [8]:
#viewing mutational ga df
mut_ga

,location,variant,median_ga,ga_upper_80,ga_lower_80
0,USA,B.1.1.529,1.0,1.0,1.0
1,USA,BA.1,1.0,1.0,1.0
2,USA,BA.1.1,1.0,1.0,1.0
3,USA,BA.1.1.1,1.0,1.0,1.0
4,USA,BA.1.1.10,1.0,1.0,1.0
...,...,...,...,...,...
173,USA,BQ.1.14,1.0,1.0,1.0
174,USA,BQ.1.2,1.0,1.0,1.0
175,USA,BQ.1.3,1.0,1.0,1.0
176,USA,BQ.1.6,1.0,1.0,1.0


# Assigning individual fitnesses to each mutation

In [9]:
# Grabbing fitness for each mutation, in sorted order
def mutations(samples, data):
    rows_for_tsv = []
    delta = jnp.median(samples["raw_delta"], axis=0)
    idx = (-delta).argsort()[:len(delta)]
    for i in idx:
        array = delta[i]
        rows_for_tsv.append({'mutation':data.mut_names[i], 'fitness_effect':array})
    return rows_for_tsv
table = mutations(samples, variant_frequencies)
mutations_table = pd.DataFrame(table)

In [10]:
# creating summary table with variant name, variant ga, mutation, mutation fitness
final_table = pd.merge(mut_ga, raw_muts, on="variant")
final_table = pd.merge(final_table, mutations_table, on="mutation")
final_table = final_table.drop(final_table.columns[0], axis=1)
final_table = final_table.drop(final_table.columns[2], axis=1)
final_table = final_table.drop(final_table.columns[2], axis=1)
# renaming the table
mutations_summary = final_table

In [11]:
# viewing table of mutation fitnesses
mutations_summary

,variant,median_ga,mutation,fitness_effect
0,BA.2.13,0.9664435,L452M_S,-0.008126772
1,BA.2.13.1,0.98715186,L452M_S,-0.008126772
2,BA.2.13.1,0.98715186,A75V_ORF9b,0.056104172
3,BA.2.13.1,0.98715186,T941S_S,-0.051056303


# Assigning innovation GA to variants

In [12]:
# Getting data
raw_seq = pd.read_csv("../count-data/pango_location-variant-sequence-counts.tsv", sep="\t")
# raw_variant_parents is missing a row for BA.2, so append one
raw_variant_parents = pd.concat((pd.read_csv("../count-data/pango_variant-relationships.tsv", sep="\t"),
                                pd.DataFrame({"variant": ["BA.2"], "parent": ["base"]}))).reset_index(drop=True)
variant_frequencies = ef.InnovationSequenceCounts(raw_seq, raw_variant_parents, pivot="BA.2")

In [13]:
# Defining model
mlr = ef.InnovationMLR(tau=4.2)
# Defining inference method
inference_method = ef.InferFullRank(iters=50_000, lr=4e-3, num_samples=100)
# Fitting model
posterior = inference_method.fit(mlr, variant_frequencies)
samples = posterior.samples

KeyboardInterrupt: 

In [ ]:
innovation_ga = pd.DataFrame(
    ef.posterior.get_growth_advantage(samples, posterior.data, ps=[0.8], name="USA", rel_to="BA.2")
)

In [ ]:
#viewing innovation ga df
innovation_ga

# Comparing Innovation and Mutational Growth Advantages

In [ ]:
# resolving weird formatting issues btwn csv and loading pandas df??
mut_ga.to_csv('mut_ga.tsv', sep="\t")
innovation_ga.to_csv('innov_ga.tsv', sep="\t")

In [ ]:
# loading in datasets
parent_variant_relationships = pd.read_csv("../count-data/pango_variant-relationships.tsv", sep="\t").set_index("variant")
mut_ga = pd.read_csv("mut_ga.tsv", sep="\t")
innovation_ga = pd.read_csv("innov_ga.tsv", sep="\t")

In [ ]:
# Prep to merge
def clean_and_merge(innovation_ga, mut_ga, parent_variant_relationships):
    innovation_ga = (innovation_ga
                     .rename(columns={"median_ga": "innov-ga"})
                     [["variant", "innov-ga"]])

    mut_ga = (mut_ga
              .rename(columns={"median_ga": "mut-ga"})
              [["variant", "mut-ga"]])

    ga_df = innovation_ga.merge(mut_ga, how="right").set_index("variant")
    ga_df["mut-ga"] = ga_df["mut-ga"] / ga_df.loc["BA.2.1"]["mut-ga"] # Scale GA by BA.2.1

    # Add parents
    def _get_parent(row):
        variant = row.name
        if variant in parent_variant_relationships.index:
            return parent_variant_relationships.loc[variant]
        return "NAN"

    ga_df["parent"] = ga_df.apply(_get_parent, axis=1)
    return ga_df

In [ ]:
ga_df = clean_and_merge(innovation_ga, mut_ga, parent_variant_relationships)

In [ ]:
# Look for parent for mut-parent-ga
def add_deltas(row, prefix):
    # Give up if parent not found
    if row.parent not in ga_df.index:
        return pd.Series([np.nan, np.nan],
                         index=[prefix + "-parent-ga", prefix + "-pc-ratio"])
    # Get child growth advantage
    parent_row = ga_df.loc[row.parent]
    
    # Return parent ga and ratio
    return pd.Series([parent_row[prefix + "-ga"],  row[prefix + "-ga"] / parent_row[prefix + "-ga"]],
                         index=[prefix + "-parent-ga", prefix + "-pc-ratio"])
    
ga_df[["mut-parent-ga", "mut-pc-ratio"]] = ga_df.apply(add_deltas, prefix="mut", axis=1)
ga_df[["innov-parent-ga", "innov-pc-ratio"]] = ga_df.apply(add_deltas, prefix="innov", axis=1)

In [ ]:
def is_outlier_IQR(data, col, scale=1.5):
    """
    Find wheter outliers in column col from dataframe data
    Uses IQR rule based on normal approximation.
    """

    Q3 = np.quantile(data[col], 0.75)
    Q1 = np.quantile(data[col], 0.25)
    IQR = Q3 - Q1
    lower_bound = Q1 - scale * IQR
    upper_bound = Q3 + scale * IQR
    return (data[col] < lower_bound) | (data[col] > upper_bound)

    
ga_df["innov-mut-ratio"] = ga_df["innov-ga"] / ga_df["mut-ga"]
ga_df["log-innov-mut-ratio"] = np.log(ga_df["innov-ga"] / ga_df["mut-ga"])
ga_df["outlier"] = is_outlier_IQR(ga_df, "log-innov-mut-ratio") # log-innov-mut-ratio ~ Normal()

In [ ]:
# viewing ga_df
ga_df

## Plotting mutational ga vs. innovation ga

Each datapoint is a variant. Red datapoints are variants in which the rato between innovation/mutational ga is considered an outlier. Therefore, red variants might indicate epistasis where blue datapoint variants would indicate no epistasis. 

In [ ]:
# Coloring by outlier
plt.scatter(ga_df["innov-ga"], ga_df["mut-ga"], 
            color=["pink" if value else "lightblue" for value in ga_df.outlier], 
            ec="k")
plt.plot([0.5, 2.5], [0.5, 2.5], color="k", linestyle="--")
plt.xlabel("Innovation GA")
plt.ylabel("Mutation GA")

# Looking at Outliers

In [ ]:
# Defining outliers
outliers = ga_df[ga_df.outlier & (ga_df["mut-pc-ratio"] < 0.5)]

In [ ]:
# Looking at outliers in bottom right with low ga
outliers

# Verifying Outlier Mutational GA Assignments

Given that:

    relative_fitness_variant = sum(mutational_fitness for mutation in variant)
    mutational_ga_variant = exp(relative_fitness_variant * generation_time)
    generation_time is fixed at 4.2

The calculated_ga variable for each variant is assigned using the equation above. 

In [ ]:
# compiling outlier summary table with indv fitnesses and mutational ga

# just looking at outliers for now
outlier_variants = ga_df[ga_df["outlier"]].index
# group by variants
grouped = mutations_summary.groupby('variant')
# final df
rows_for_tsv = []
for name, group in grouped:
    if name in outlier_variants:
        variant_dict = {}
        mutations = group['mutation']
        variant_dict['variant'] = name
        mut_ga = group['median_ga'].iloc[0]
        variant_dict['mut_ga'] = mut_ga
        cum_sum = 0
        for m in mutations:
            # grabbing the fitness effect  of each mutation
            fitness_effect = group.loc[group['mutation'] == m, 'fitness_effect'].iloc[0]
            variant_dict[m] = fitness_effect
            # summing up all mutation fitnesses
            cum_sum += fitness_effect
        # multiply by generation time and raise e to this number
        calculated_ga = math.exp(cum_sum * 4.2)
        variant_dict['calculated_ga'] = calculated_ga
        variant_dict['match'] = (mut_ga == calculated_ga)
        rows_for_tsv.append(variant_dict)
resulting_table = pd.DataFrame(rows_for_tsv)      

In [ ]:
resulting_table

In [ ]:
#aesthetic changes to the summary df

# moving mut_ga to end 
cols_at_end = ['calculated_ga', 'mut_ga', 'match']
resulting_table = resulting_table[[c for c in resulting_table if c not in cols_at_end] 
        + [c for c in cols_at_end if c in resulting_table]]
# concatenating innovation ga column info 
trimmed = ga_df[ga_df["outlier"]].drop(columns=['mut-ga','parent','mut-parent-ga','mut-pc-ratio','innov-parent-ga','innov-pc-ratio','innov-mut-ratio','log-innov-mut-ratio','outlier'])
merged = pd.merge(resulting_table, trimmed, on="variant", how="left")
resulting_table = merged

In [ ]:
# viewing mutational fitnesses for outliers
resulting_table

In [ ]:
# saving summary table to csv
resulting_table.to_csv('reformatted_table.tsv', sep='\t')

## Outstanding Questions

The calculated ga and assigned mutational ga should match for each variant. As seen above, none of them do. What is going on?